# Automated Feature Engineering
Use the featuretools library.


In [1]:
# Uncomment and run if kernel does not already have featuretools
!pip install featuretools --user

    100% |████████████████████████████████| 174kB 145kB/s 


In [2]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft
# matplotlit and seaborn for visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 22
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

/home/kke1/.local/lib/python2.7/site-packages/pandas/core/nanops.py:39: UserWarning: The installed version of bottleneck 0.7.0 is not supported in pandas and will be not be used
The minimum supported version is 1.0.0

  ver=ver, min_ver=_MIN_BOTTLENECK_VERSION), UserWarning)


## Dataset

The data is provided by [Home Credit](http://www.homecredit.net/about-us.aspx), a service dedicated to provided lines of credit (loans) to the unbanked population. 

There are 7 different data files:

* __application_train/application_test__: the main training and testing data with information about each loan application at Home Credit. Every loan has its own row and is identified by the `SK_ID_CURR`. The training application data comes with the `TARGET` with indicating 0: the loan was repaid and 1: the loan was not repaid. 
* __bureau__: data concerning client's previous credits from other financial institutions. Each previous credit has its own row in bureau and is identified by the `SK_ID_BUREAU`, Each loan in the application data can have multiple previous credits.
* __bureau_balance__: monthly data about the previous credits in bureau. Each row is one month of a previous credit, and a single previous credit can have multiple rows, one for each month of the credit length. 
* __previous_application__: previous applications for loans at Home Credit of clients who have loans in the application data. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature `SK_ID_PREV`. 
* __POS_CASH_BALANCE__: monthly data about previous point of sale or cash loans clients have had with Home Credit. Each row is one month of a previous point of sale or cash loan, and a single previous loan can have many rows.
* __credit_card_balance__: monthly data about previous credit cards clients have had with Home Credit. Each row is one month of a credit card balance, and a single credit card can have many rows.
* __installments_payment__: payment history for previous loans at Home Credit. There is one row for every made payment and one row for every missed payment. 

The diagram below (provided by Home Credit) shows how the tables are related. This will be very useful when we need to define relationships in featuretools. 

![image](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)


In [9]:
# Read in the datasets and limit to the first 1000 rows (sorted by SK_ID_CURR) 
# This allows us to actually see the results in a reasonable amount of time! 
app_train = pd.read_csv('../data/credit/application_train.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
app_test = pd.read_csv('../data/credit/application_test.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
bureau = pd.read_csv('../data/credit/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('../data/credit/bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index(drop = True).loc[:1000, :]
cash = pd.read_csv('../data/credit/POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('../data/credit/credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('../data/credit/previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('../data/credit/installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]

In [5]:
# Add identifying column
app_train['set'] = 'train'
app_test['set'] = 'test'
app_test["TARGET"] = np.nan

# Append the dataframes
app = app_train.append(app_test, ignore_index = True)

# Featuretools Basics
### Entities and Entitysets

An [EntitySet](https://docs.featuretools.com/loading_data/using_entitysets.html) is a collection of tables and the relationships between them. This can be thought of a data structute with its own methods and attributes. Using an EntitySet allows us to group together multiple tables and manipulate them much quicker than individual tables. 

First we'll make an empty entityset named clients to keep track of all the data.

In [6]:
# Entity set with id applications
es = ft.EntitySet(id = 'clients')

Now we define each entity, or table of data. We need to pass in an index if the data has one or `make_index = True` if not. Featuretools will automatically infer the types of variables, but we can also change them if needed. For intstance, if we have a categorical variable that is represented as an integer we might want to let featuretools know the right type.

In [11]:
# Entities with a unique index
es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 'SK_ID_CURR')

es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')

es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV')

# Entities that do not have a unique index
es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, 
                              make_index = True, index = 'bureaubalance_index')

es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index')

es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                              make_index = True, index = 'installments_index')

es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index')

### Relationships
We have one-to-many relationship in this project.

In [12]:
print('Parent: app, Parent Variable: SK_ID_CURR\n\n', app.iloc[:, 111:115].head())
print('\nChild: bureau, Child Variable: SK_ID_CURR\n\n', bureau.iloc[10:30, :4].head())

('Parent: app, Parent Variable: SK_ID_CURR\n\n',    SK_ID_CURR  TARGET  TOTALAREA_MODE WALLSMATERIAL_MODE
0      100002     1.0          0.0149       Stone, brick
1      100003     0.0          0.0714              Block
2      100004     0.0             NaN                NaN
3      100006     0.0             NaN                NaN
4      100007     0.0             NaN                NaN)
('\nChild: bureau, Child Variable: SK_ID_CURR\n\n',     SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY
10      100002       6158905        Closed      currency 1
11      100002       6158906        Closed      currency 1
12      100002       6158907        Closed      currency 1
13      100002       6158908        Closed      currency 1
14      100002       6158909        Active      currency 1)


The `SK_ID_CURR` "100002" has one row in the parent table and multiple rows in the child. 

Two tables are linked via a shared variable. The `app` and `bureau` dataframe are linked by the `SK_ID_CURR` variable while the `bureau` and `bureau_balance` dataframes are linked with the `SK_ID_BUREAU`. Defining the relationships is relatively straightforward, and the diagram provided by the competition is helpful for seeing the relationships. For each relationship, we need to specify the parent variable and the child variable. Altogether, there are a total of 6 relationships between the tables. Below we specify all six relationships and then add them to the EntitySet.

In [14]:
# Relationship between app and bureau
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])

# Relationship between bureau and bureau balance
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

# Relationship between current app and previous apps
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])

# Relationships between previous apps and cash, installments, and credit
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [15]:
# Add in the defined relationships
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])
# Print out the EntitySet
es

Entityset: clients
  Entities:
    app [Rows: 2002, Columns: 123]
    cash [Rows: 1001, Columns: 9]
    bureau_balance [Rows: 1001, Columns: 4]
    credit [Rows: 1001, Columns: 24]
    bureau [Rows: 1001, Columns: 17]
    installments [Rows: 1001, Columns: 9]
    previous [Rows: 1001, Columns: 37]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

Slightly advanced note: we need to be careful to not create a [diamond graph](https://en.wikipedia.org/wiki/Diamond_graph) where there are multiple paths from a parent to a child. If we directly link `app` and `cash` via `SK_ID_CURR`; `previous` and `cash` via `SK_ID_PREV`; and `app` and `previous` via `SK_ID_CURR`, then we have created two paths from `app` to `cash`. This results in ambiguity, so the approach we have to take instead is to link `app` to `cash` through `previous`. We establish a relationship between `previous` (the parent) and `cash` (the child) using `SK_ID_PREV`. Then we establish a relationship between `app` (the parent) and `previous` (now the child) using `SK_ID_CURR`. Then featuretools will be able to create features on `app` derived from both `previous` and `cash` by stacking multiple primitives. 

All entities in the entity can be related to each other. In theory this allows us to calculate features for any of the entities, but in practice, we will only calculate features for the `app` dataframe since that will be used for training/testing. 

# Feature Primitives

A [feature primitive](https://docs.featuretools.com/automated_feature_engineering/primitives.html) is an operation applied to a table or a set of tables to create a feature.Feature primitives fall into two categories:

* __Aggregation__: function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum previous loan amount for each client. An aggregation works across multiple tables using relationships between tables.
* __Transformation__: an operation applied to one or more columns in a single table. An example would be taking the absolute value of a column, or finding the difference between two columns in one table.


In [16]:
# List the primitives in a dataframe
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,std,aggregation,Finds the standard deviation of a numeric feature ignoring null values.
1,count,aggregation,Counts the number of non null values.
2,all,aggregation,Test if all values are 'True'.
3,last,aggregation,Returns the last value.
4,n_most_common,aggregation,Finds the N most common elements in a categorical feature.
5,min,aggregation,Finds the minimum non-null value of a numeric feature.
6,trend,aggregation,Calculates the slope of the linear trend of variable overtime.
7,max,aggregation,Finds the maximum non-null value of a numeric feature.
8,skew,aggregation,Computes the skewness of a data set.
9,median,aggregation,Finds the median value of any feature with well-ordered values.


In [17]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,description
19,and,transform,"For two boolean values, determine if both values are 'True'."
20,cum_sum,transform,Calculates the sum of previous values of an instance for each value in a time-dependent entity.
21,months,transform,Transform a Timedelta feature into the number of months.
22,subtract,transform,Creates a transform feature that subtracts two features.
23,divide,transform,Creates a transform feature that divides two features.
24,cum_min,transform,Calculates the min of previous values of an instance for each value in a time-dependent entity.
25,is_null,transform,"For each value of base feature, return 'True' if value is null."
26,years,transform,Transform a Timedelta feature into the number of years.
27,multiply,transform,Creates a transform feature that multplies two features.
28,second,transform,Transform a Datetime feature into the second.


# Deep Feature Synthesis

Deep Feature Synthesis (DFS) is the process featuretools uses to make new features. DFS stacks feature primitives to form features with a "depth" equal to the number of primitives. For example, if we take the maximum value of a client's previous loans (say `MAX(previous.loan_amount)`), that is a "deep feature" with a depth of 1. To create a feature with a depth of two, we could stack primitives by taking the maximum value of a client's average montly payments per previous loan (such as `MAX(previous(MEAN(installments.payment)))`). The [original paper on automated feature engineering using deep feature synthesis](https://dai.lids.mit.edu/wp-content/uploads/2017/10/DSAA_DSM_2015.pdf) is worth a read. 

To perform DFS in featuretools, we use the `dfs`  function passing it an `entityset`, the `target_entity` (where we want to make the features), the `agg_primitives` to use, the `trans_primitives` to use and the `max_depth` of the features. Here we will use the default aggregation and transformation primitives,  a max depth of 2, and calculate primitives for the `app` entity. Because this process is computationally expensive, we can run the function using `features_only = True` to return only a list of the features and not calculate the features themselves. This can be useful to look at the resulting features before starting an extended computation.

### DFS with Default Primitives

In [18]:
# Default primitives from featuretools
default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
default_trans_primitives =  ["day", "year", "month", "weekday", "haversine", "numwords", "characters"]

# DFS with specified primitives
feature_names = ft.dfs(entityset = es, target_entity = 'app',
                       trans_primitives = default_trans_primitives,
                       agg_primitives=default_agg_primitives, 
                       max_depth = 2, features_only=True)

print('%d Total Features' % len(feature_names))

1697 Total Features


If you are interested in running this call on the entire dataset and making the features, I wrote a script [for that here](https://www.kaggle.com/willkoehrsen/feature-engineering-using-feature-tools). Unfortunately, this will not run in a Kaggle kernel due to the computational expense of the operation. Using a computer with 64GB of ram, this function call took around 24 hours (I don't think I'm technically breaking the rules of my university's high powered computing center). I have made the entire dataset available [here](https://www.kaggle.com/willkoehrsen/home-credit-default-risk-feature-tools/data) in the file called `feature_matrix.csv`. 

To generate a subset of the features, run the code cell below.

In [19]:
# DFS with default primitives
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'app',
                                       trans_primitives = default_trans_primitives,
                                       agg_primitives=default_agg_primitives, 
                                        max_depth = 2, features_only=False, verbose = True)

pd.options.display.max_columns = 1700
feature_matrix.head(10)

Built 1697 features
Elapsed: 00:51 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


FLAG_DOCUMENT_18  NAME_HOUSING_TYPE  LANDAREA_AVG  \
SK_ID_CURR                                                      
100001                     0  House / apartment           NaN   
100002                     0  House / apartment        0.0369   
100003                     0  House / apartment        0.0130   
100004                     0  House / apartment           NaN   
100005                     0  House / apartment           NaN   
100006                     0  House / apartment           NaN   
100007                     0  House / apartment           NaN   
100008                     0  House / apartment           NaN   
100009                     0  House / apartment           NaN   
100010                     0  House / apartment           NaN   

            BASEMENTAREA_MEDI  AMT_REQ_CREDIT_BUREAU_YEAR  \
SK_ID_CURR                                                  
100001                 0.0590                         0.0   
100002                 0.0369                         1.0   
100003                 0.0529                         0.0   
100004                    NaN                         0.0   
100005                    NaN                         3.0   
100006                    NaN                         NaN   
100007                    NaN                         0.0   
100008                    NaN                         1.0   
100009                    NaN                         2.0   
100010                    NaN                         0.0   

            DAYS_LAST_PHONE_CHANGE  EXT_SOURCE_2  APARTMENTS_MEDI  \
SK_ID_CURR                                                          
100001                     -1740.0      0.789654           0.0666   
100002                     -1134.0      0.262949           0.0250   
100003                      -828.0      0.622246           0.0968   
100004                      -815.0      0.555912              NaN   
100005                         0.0      0.291656              NaN   
100006                      -617.0      0.650442              NaN   
100007                     -1106.0      0.322738              NaN   
100008                     -2536.0      0.354225              NaN   
100009                     -1562.0      0.724000              NaN   
100010                     -1070.0      0.714279              NaN   

            AMT_ANNUITY  REGION_RATING_CLIENT_W_CITY  FLAG_DOCUMENT_13  \
SK_ID_CURR                                                               
100001          20560.5                            2                 0   
100002          24700.5                            2                 0   
100003          35698.5                            1                 0   
100004           6750.0                            2                 0   
100005          17370.0                            2                 0   
100006          29686.5                            2                 0   
100007          21865.5                            2                 0   
100008          27517.5                            2                 0   
100009          41301.0                            2                 0   
100010          42075.0                            3                 0   

            TOTALAREA_MODE EMERGENCYSTATE_MODE  FLAG_DOCUMENT_17  \
SK_ID_CURR                                                         
100001              0.0392                  No                 0   
100002              0.0149                  No                 0   
100003              0.0714                  No                 0   
100004                 NaN                 NaN                 0   
100005                 NaN                 NaN                 0   
100006                 NaN                 NaN                 0   
100007                 NaN                 NaN                 0   
100008                 NaN                 NaN                 0   
100009                 NaN                 NaN                 0   
100010                 NaN      

In [20]:
feature_names[-20:]

[<Feature: MIN(previous.MAX(credit.AMT_DRAWINGS_ATM_CURRENT))>,
 <Feature: SUM(previous.MIN(credit.CNT_INSTALMENT_MATURE_CUM))>,
 <Feature: MAX(previous.MIN(credit.MONTHS_BALANCE))>,
 <Feature: MIN(previous.SUM(credit.AMT_RECIVABLE))>,
 <Feature: SKEW(previous.SUM(credit.SK_DPD))>,
 <Feature: SUM(previous.MAX(cash.SK_ID_CURR))>,
 <Feature: MIN(previous.MEAN(credit.AMT_DRAWINGS_OTHER_CURRENT))>,
 <Feature: STD(previous.MIN(credit.MONTHS_BALANCE))>,
 <Feature: SUM(bureau.MAX(bureau_balance.MONTHS_BALANCE))>,
 <Feature: SUM(previous.MEAN(credit.AMT_PAYMENT_TOTAL_CURRENT))>,
 <Feature: MAX(previous.STD(credit.AMT_INST_MIN_REGULARITY))>,
 <Feature: STD(previous.SKEW(cash.CNT_INSTALMENT))>,
 <Feature: MEAN(previous.SUM(installments.DAYS_ENTRY_PAYMENT))>,
 <Feature: MAX(previous.SUM(installments.DAYS_INSTALMENT))>,
 <Feature: SUM(previous.STD(installments.AMT_PAYMENT))>,
 <Feature: SKEW(previous.MEAN(installments.DAYS_ENTRY_PAYMENT))>,
 <Feature: SKEW(previous.MEAN(credit.AMT_RECIVABLE))>,
 <

### DFS with Selected Aggregation Primitives

With featuretools, we were able to go from 121 original features to almost 1700 in a few lines of code.  When I did feature engineering by hand, it took about 12 hours to create a comparable size dataset. However, while we get a lot of features in featuretools, this function call is not very well-informed. We simply used the default aggregations without thinking about which ones are "important" for the problem. We end up with a lot of features, but they are probably not all relevant to the problem. Too many irrelevant features can decrease performance by drowning out the important features (related to the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality))

The next call we make will specify a smaller set of features. We still are not using much domain knowledge, but this feature set will be more manageable. The next step from here is improving the features we actually build and performing feature selection.

In [21]:
# Specify the aggregation primitives
feature_matrix_spec, feature_names_spec = ft.dfs(entityset = es, target_entity = 'app',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean', 'mode'], 
                                                 max_depth = 2, features_only = False, verbose = True)

Built 884 features
Elapsed: 00:23 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


That "only" gives us 884 features (and takes about 12 hours to run on the complete dataset). 

In [22]:
pd.options.display.max_columns = 1000
feature_matrix_spec.head(10)

FLAG_DOCUMENT_18  ENTRANCES_MEDI  HOUR_APPR_PROCESS_START  \
SK_ID_CURR                                                              
100001                     0          0.1379                       18   
100002                     0          0.0690                       10   
100003                     0          0.0345                       11   
100004                     0             NaN                        9   
100005                     0             NaN                        9   
100006                     0             NaN                       17   
100007                     0             NaN                       11   
100008                     0             NaN                       16   
100009                     0             NaN                       16   
100010                     0             NaN                       16   

            DEF_60_CNT_SOCIAL_CIRCLE  REGION_POPULATION_RELATIVE  \
SK_ID_CURR                                                         
100001                           0.0                    0.018850   
100002                           2.0                    0.018801   
100003                           0.0                    0.003541   
100004                           0.0                    0.010032   
100005                           0.0                    0.035792   
100006                           0.0                    0.008019   
100007                           0.0                    0.028663   
100008                           0.0                    0.035792   
100009                           0.0                    0.035792   
100010                           0.0                    0.003122   

            FLOORSMIN_MEDI  TARGET  DAYS_EMPLOYED  BASEMENTAREA_MEDI  \
SK_ID_CURR                                                             
100001                 NaN     NaN          -2329             0.0590   
100002              0.1250     1.0           -637             0.0369   
100003              0.3333     0.0          -1188             0.0529   
100004                 NaN     0.0           -225                NaN   
100005                 NaN     NaN          -4469                NaN   
100006                 NaN     0.0          -3039                NaN   
100007                 NaN     0.0          -3038                NaN   
100008                 NaN     0.0          -1588                NaN   
100009                 NaN     0.0          -3130                NaN   
100010                 NaN     0.0           -449                NaN   

            ENTRANCES_MODE  AMT_REQ_CREDIT_BUREAU_YEAR FLAG_OWN_REALTY  \
SK_ID_CURR                                                               
100001              0.1379                         0.0               Y   
100002              0.0690                         1.0               Y   
100003              0.0345                         0.0               N   
100004                 NaN                         0.0               Y   
100005                 NaN                         3.0               Y   
100006                 NaN                         NaN               Y   
100007                 NaN                         0.0               Y   
100008                 NaN                         1.0               Y   
100009                 NaN                         2.0               Y   
100010                 NaN                         0.0               Y   

            AMT_CREDIT  DAYS_LAST_PHONE_CHANGE  LIVINGAREA_AVG  EXT_SOURCE_2  \
SK_ID_CURR                                                                     
100001        568800.0                 -1740.0          0.0505      0.789654   
100002        406597.5                 -1134.0          0.0190      0.262949   
100003       1293502.5                  -828.0          0.0549      0.622246   
100004        135000.0                  -815.0             NaN      0.555912   
100005        222768.0                     0.0             NaN      0.291656   
1000